In [1]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from fpdf import FPDF
import re
from datetime import datetime

In [2]:
# Ensure required NLTK data packages are downloaded
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')

# Initialize Sentiment Analyzer
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\5A_Traders\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\5A_Traders\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\5A_Traders\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\5A_Traders\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\5A_Traders\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
# Sample email dataset
emails = [
    {"subject": "Urgent: Complete the budget report", "sender": "projectlead@example.com", "body": "Please complete the budget report by EOD today."},
    {"subject": "Reminder: Team meeting", "sender": "hrmanager@example.org", "body": "Join the team meeting tomorrow at 10 AM."},
    {"subject": "Follow-up: Send the client proposal", "sender": "accountmanager@example.com", "body": "Send the updated proposal to the client by next week."},
    {"subject": "Important: Update the project plan", "sender": "teamlead@example.net", "body": "Update the project plan with the latest milestones and share it with the team."},
    {"subject": "Task: Review the quarterly performance", "sender": "adminoffice@example.com", "body": "Review the performance metrics for the last quarter and prepare a summary."},
    {"subject": "Alert: Fix the server issue", "sender": "it_support@example.com", "body": "The production server is down. Please fix the issue immediately."},
    {"subject": "Request: Schedule a meeting with the vendor", "sender": "eventcoordinator@example.com", "body": "Coordinate with the vendor and schedule a meeting next week."},
    {"subject": "Note: Prepare for the client presentation", "sender": "trainingdept@example.org", "body": "Work on the slides for the upcoming client presentation."},
    {"subject": "Reminder: Submit the expense reports", "sender": "financehead@example.com", "body": "Ensure all expense reports are submitted by the end of this week."},
    {"subject": "Task: Organize the team outing", "sender": "marketinglead@example.com", "body": "Plan and organize the team outing for next month."}
]

In [4]:
def sanitize_text(text):
    """Replace unsupported Unicode characters with ASCII equivalents."""
    replacements = {
        "\u2018": "'",  # Left single quotation mark
        "\u2019": "'",  # Right single quotation mark
        "\u201C": "\"",  # Left double quotation mark
        "\u201D": "\"",  # Right double quotation mark
        "\u2013": "-",  # En dash
        "\u2014": "-",  # Em dash
    }
    for unicode_char, ascii_char in replacements.items():
        text = text.replace(unicode_char, ascii_char)
    return text

def analyze_priority(subject, body):
    """Analyze priority based on email subject and body using advanced sentiment analysis."""
    combined_text = subject + " " + body
    sentiment_scores = sia.polarity_scores(combined_text)
    sentiment_score = sentiment_scores['compound']
    neg_score = sentiment_scores['neg']
    pos_score = sentiment_scores['pos']

    # Assign priority based on sentiment and keywords
    if "urgent" in subject.lower() or "alert" in subject.lower() or "deadline" in subject.lower():
        return 1
    elif "important" in subject.lower():
        return 2
    elif "reminder" in subject.lower():
        return 3
    elif neg_score > 0.4:
        return 1  # High priority for strong negative sentiment
    elif pos_score > 0.4:
        return 4  # Low priority for strong positive sentiment
    elif sentiment_score < -0.3:
        return 2  # Elevated priority for negative sentiment
    elif sentiment_score > 0.3:
        return 4  # Reduced priority for positive sentiment
    else:
        return 5  # Default low priority

def extract_tasks_from_emails(emails):
    """Extract tasks and priorities from email content."""
    tasks = []

    for email in emails:
        priority = analyze_priority(email['subject'], email['body'])

        # Extract tasks from the email body (short and precise)
        sentences = sent_tokenize(email['body'])
        task = sentences[0] if sentences else email['subject']  # Use the first sentence of the body or subject
        task = ' '.join(task.split()[:10])  # Limit task to 10 words

        tasks.append({
            "task": task,
            "Email Sender": email['sender'],
            "Email Subject": email['subject'],
            "Priority": priority
        })

    # Sort tasks by priority
    tasks = sorted(tasks, key=lambda x: x['Priority'])
    return tasks

def save_tasks_to_pdf(tasks, filename="todo_list.pdf"):
    """Save tasks to a PDF file."""
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    pdf.cell(200, 10, txt="To-Do List", ln=True, align='C')

    # Add the current date
    current_date = datetime.now().strftime("%d/%m/%Y")
    pdf.cell(200, 10, txt=f"Date: {current_date}", ln=True, align='C')

    pdf.ln(10)

    for task_entry in tasks:
        pdf.cell(0, 10, txt=f"Priority: {task_entry['Priority']}", ln=True)
        pdf.cell(0, 10, txt=f"Task: {sanitize_text(task_entry['task'])}", ln=True)
        pdf.cell(0, 10, txt=f"Email Sender: {sanitize_text(task_entry['Email Sender'])}", ln=True)
        pdf.cell(0, 10, txt=f"Email Subject: {sanitize_text(task_entry['Email Subject'])}", ln=True)
        pdf.cell(0, 10, txt=f"___________________________________________________________", ln=True)
        pdf.ln(5)

    pdf.output(filename)
    print(f"To-Do list saved to {filename}")

# Extract tasks from emails
tasks = extract_tasks_from_emails(emails)

# Save the tasks to a PDF
save_tasks_to_pdf(tasks)

To-Do list saved to todo_list.pdf
